In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./datasets/good_features.csv',index_col='Id')

In [3]:
train_dtypes = pd.read_csv('./datasets/train_agg_dtypes.csv')
train_dtypes.set_index('Attribute',inplace=True)
train_dtypes.drop('Unnamed: 0',axis=1,inplace=True)
train_dtypes.rename_axis(None,inplace=True)

In [4]:
replacement_dict = pd.read_csv('./datasets/data_instructions.csv')
replacement_dict.set_index('Unnamed: 0',inplace=True)
replacement_dict.rename_axis(None,inplace=True)
training_nulls = replacement_dict.index.tolist()

In [5]:
test_df = pd.read_csv('./datasets/test.csv',index_col='Id')

## Remove Nulls

In [6]:
def test_null_remover(raw):
    [raw[each].fillna('None', inplace=True) for each in training_nulls[0:5]];
    
    raw['Garage Yr Blt'] = raw['Garage Yr Blt'].fillna(value=raw['Year Built'])
    
    area_etc_strings = ['SF','Area','Frontage', 'Bath', 'Cars']
    gar_bsmt_strings = ['Garage','Bsmt', 'Type']


    for each in training_nulls[5:]:
        for string in area_etc_strings:
            if string in each:
                raw[each].fillna(0,inplace=True)

    for each in training_nulls[6:]:
        for string in gar_bsmt_strings:
            if string in each:
                raw[each].fillna('None',inplace=True)
                
    null_or_not = pd.DataFrame(raw.isnull().sum().sort_values(ascending=False) == 1)
    null_features = null_or_not.loc[null_or_not[0]==1].index.tolist()
    
    for each in null_features:
        if train_dtypes['D_Type'][each] == 'object':
            raw[each].fillna('None')
        elif train_dtypes['D_Type'][each] == 'int64':
            raw[each].fillna(0)
        elif train_dtypes['D_Type'][each] == 'float64':
            raw[each].fillna(0)
    
    return raw

## Engineer Features

In [7]:
def create_man_features(df):
    
    starter = df
    
    starter['Overall Qual_obj'] = starter['Overall Qual'].astype(object)
    starter['Overall Cond_obj'] = starter['Overall Cond'].astype(object)
    starter['Has_Pool'] = starter['Pool Area'] > 0
    starter['Has_Garage'] = starter['Garage Area'] > 0
    SF_cols = [each for each in starter.columns if 'SF' in each]
    starter['Total SF'] = starter[SF_cols].sum(axis=1)

    starter['Total Living Space'] = starter['Total SF'] + starter['Gr Liv Area']
    starter['HasBasement'] = starter['Total Bsmt SF'] > 0
    starter['TotRms AbvGrd_morethan6'] = starter['TotRms AbvGrd'] > 6
    
    starter['Total Living Space_morethan10k'] = starter['Total Living Space'] > 10000
    starter.loc[starter['Total Living Space'] > 15000]

    starter['MS SubClass'] = starter['MS SubClass'].astype(object)

    starter['Qual_LivingArea'] = (starter['Total Living Space'] / 1000) * (starter['Overall Qual'])

    starter['Qual_LivingArea_veryhigh'] = starter['Qual_LivingArea'] > 75
    starter['Qual_LivingArea_high'] = (starter['Qual_LivingArea'] > 57) & (starter['Qual_LivingArea'] < 75)
    
    starter['NewlyBuilt'] = starter['Year Built'] > 2005
    starter['Built2000s'] = starter['Year Built'] > 2000
    starter['Built90s'] = (starter['Year Built'] > 1989) & (starter['Year Built'] < 2000)
    
    starter['NewlyRemodeled'] = starter['Year Built'] > 2005
    starter['Remodeled2000s'] = starter['Year Built'] > 2005

    return starter

In [8]:
def match_w_dummies(df):
    
    categorical = df.select_dtypes(include=['object','bool'])
    numerical = df.select_dtypes(include=['float64','int'])
    
    cat_dummies = pd.get_dummies(categorical)
    test_w_dummies = pd.concat([numerical, cat_dummies],axis=1)
    
    test_w_dummies.drop(['PID'],axis=1,inplace=True)
    
    not_in_test = []
    
    temptrain = pd.read_csv('./datasets/good_features.csv',index_col='Id')
    
    for each in temptrain.columns:
        if each not in test_w_dummies.columns:
            not_in_test.append(each)
            print(each + ' was not in test data')    
    
    not_in_test.remove('SalePrice')
    
    for each in not_in_test:
        train.drop(each,axis=1,inplace=True)
        
    good_cols = train.columns.tolist()
    good_cols.remove('SalePrice')
    
    #use the code below to remove any optional columns before modeling
    
    #good_cols.remove([''])
    
    return test_w_dummies[good_cols]

In [9]:
test_df = test_null_remover(test_df)
test_df = create_man_features(test_df)
test_df = match_w_dummies(test_df)

Bsmt Cond_Ex was not in test data
Bsmt Cond_Po was not in test data
Condition 2_Artery was not in test data
Condition 2_RRAn was not in test data
Condition 2_RRNn was not in test data
Electrical_Mix was not in test data
Exterior 1st_CBlock was not in test data
Exterior 1st_ImStucc was not in test data
Exterior 1st_Stone was not in test data
Exterior 2nd_Stone was not in test data
Functional_Sal was not in test data
Functional_Sev was not in test data
Garage Qual_Ex was not in test data
Has Fireplace was not in test data
Heating QC_Po was not in test data
Heating_OthW was not in test data
Heating_Wall was not in test data
MS SubClass_150 was not in test data
MS Zoning_A (agr) was not in test data
Misc Feature_TenC was not in test data
Neighborhood_GrnHill was not in test data
Neighborhood_Landmrk was not in test data
Overall Qual_obj_1 was not in test data
Pool QC_Fa was not in test data
Pool QC_Gd was not in test data
Roof Matl_Membran was not in test data
SalePrice was not in test dat

In [10]:
test_df.head()

,3Ssn Porch,Alley_Grvl,Alley_None,Alley_Pave,Bedroom AbvGr,Bldg Type_1Fam,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,...,Street_Pave,TotRms AbvGrd,Total Bsmt SF,Total Living Space_morethan10k,Utilities_AllPub,Utilities_NoSewr,Wood Deck SF,Year Built,Year Remod/Add,Yr Sold
Id,,,,,,,,,,,,,,,,,,,,,
2658,0,1,0,0,4,0,1,0,0,0,...,1,9,1020,False,1,0,0,1910,1950,2006
2718,0,0,1,0,6,0,0,1,0,0,...,1,10,1967,False,1,0,170,1977,1977,2006
2414,0,0,1,0,3,1,0,0,0,0,...,1,7,654,False,1,0,100,2006,2006,2006
1989,0,0,1,0,2,1,0,0,0,0,...,1,5,968,False,1,0,0,1923,2006,2007
625,0,0,1,0,3,1,0,0,0,0,...,1,6,1394,False,1,0,0,1963,1963,2009


In [11]:
train.head()

,3Ssn Porch,Alley_Grvl,Alley_None,Alley_Pave,Bedroom AbvGr,Bldg Type_1Fam,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,...,Street_Pave,TotRms AbvGrd,Total Bsmt SF,Total Living Space_morethan10k,Utilities_AllPub,Utilities_NoSewr,Wood Deck SF,Year Built,Year Remod/Add,Yr Sold
Id,,,,,,,,,,,,,,,,,,,,,
109,0,0,1,0,3,1,0,0,0,0,...,1,6,725.0,False,1,0,0,1976,2005,2010
544,0,0,1,0,4,1,0,0,0,0,...,1,8,913.0,False,1,0,0,1996,1997,2009
153,0,0,1,0,3,1,0,0,0,0,...,1,5,1057.0,False,1,0,0,1953,2007,2010
318,0,0,1,0,3,1,0,0,0,0,...,1,7,384.0,False,1,0,100,2006,2007,2010
255,0,0,1,0,3,1,0,0,0,0,...,1,6,676.0,False,1,0,0,1900,1993,2010


In [12]:
test_df.to_csv('./datasets/kaggle_preprocessed.csv')

In [13]:
train.to_csv('./datasets/good_cols.csv')